In [1]:
# Part 1: Holy hell this was hard.

In [35]:
import numpy as np
import itertools
inputs = """##..#.#.
#####.##
#######.
#..#..#.
#.#...##
..#....#
....#..#
..##.#..""".split("\n")

In [36]:
slice_ = np.stack([[char for char in input_] for input_ in inputs])

In [4]:
z_to_slice = {0: slice_}

In [5]:
def update_layer(z):
    prev, next_ = z-1, z+1
    curr_z = z_to_slice[z]
    padded_z = np.pad(curr_z, 1, constant_values=".")
    new_z = padded_z.copy()
    
    if z == min(z_to_slice.keys()):
        prev_slice = np.full(curr_z.shape, ".")
    else:
        prev_slice = z_to_slice[prev]
        
    if z == max(z_to_slice.keys()):
        next_slice = np.full(curr_z.shape, ".")
    else:
        next_slice = z_to_slice[next_]
        
    
    prev_slice = np.pad(prev_slice, 1, constant_values=".")
    next_slice = np.pad(next_slice, 1, constant_values=".")
    
    for row in range(padded_z.shape[0]):
        for col in range(padded_z.shape[1]): 
            
            # Helps deal with edges    
            row_is_zero = row == 0
            row_is_last = row == padded_z.shape[0]-1 
            col_is_zero = col == 0
            col_is_last = col == padded_z.shape[1]-1

            num_active_this = np.sum(padded_z[row-(not row_is_zero):row+1+(not row_is_last), 
                                              col-(not col_is_zero): col+1+(not col_is_last)] == "#") \
                                - (padded_z[row][col] == "#")

            num_active_prev =  np.sum(prev_slice[row-(not row_is_zero):row+1+(not row_is_last), 
                                               col-(not col_is_zero): col+1+(not col_is_last)] == "#")
            num_active_next =  np.sum(next_slice[row-(not row_is_zero):row+1+(not row_is_last), 
                                                 col-(not col_is_zero): col+1+(not col_is_last)] == "#")
            
            
            total_active = num_active_this + num_active_prev + num_active_next  
            if padded_z[row][col] == "#":
                if total_active == 2 or total_active ==3:
                    new_z[row][col] = "#"
                else:
                    new_z[row][col] = "."
            else:
                if total_active == 3:
                    new_z[row][col] = "#"
                else:
                    new_z[row][col] = "."
    return new_z

In [6]:
for _ in range(6):
    new_min, new_max = min(z_to_slice.keys())-1, max(z_to_slice.keys())+1
    
    z_to_slice[new_min] = np.full(z_to_slice[0].shape, ".")
    z_to_slice[new_max] = np.full(z_to_slice[0].shape, ".")
    new_z_to_slice = {z: update_layer(z) for z in z_to_slice}
    z_to_slice = new_z_to_slice

In [7]:
total_active = 0
for z in z_to_slice:
    total_active += np.sum(z_to_slice[z] == "#")

total_active

112

In [19]:
# Part 2 Jesus christ wow

In [37]:
z_w_to_slice = {(0, 0): slice_}

In [38]:
def update_layer(z, w):
    curr_slice = z_w_to_slice[(z, w)]
    padded_slice = np.pad(curr_slice, 1, constant_values=".")
    new_slice = padded_slice.copy()
    for row in range(padded_slice.shape[0]):
        for col in range(padded_slice.shape[1]):
            
            # Deal with the edges
            row_is_zero = row == 0
            row_is_last = row == padded_slice.shape[0]-1 
            col_is_zero = col == 0
            col_is_last = col == padded_slice.shape[1]-1
            
            # Direction to go in z, w
            neighbor_slices = itertools.product([-1, 0, 1], repeat=2)
            neighbors_active = 0
            for dz, dw in neighbor_slices:
                try: 
                    padded_neighbor = np.pad(z_w_to_slice[(z+dz, w+dw)], 1, constant_values=".")
                except:
                    padded_neighbor = np.full(padded_slice.shape, ".")
                
                neighbors_active += np.sum(padded_neighbor[row-(not row_is_zero):row+1+(not row_is_last), 
                                     col-(not col_is_zero): col+1+(not col_is_last)] == "#")
                
            neighbors_active -= padded_slice[row][col] == "#"
            if padded_slice[row][col] == "#":
                if neighbors_active == 2 or neighbors_active == 3:
                    new_slice[row][col] = "#"
                else:
                    new_slice[row][col] = "."
            
            else:
                if neighbors_active == 3:
                    new_slice[row][col] = "#"
                else:
                    new_slice[row][col] = "."
                    
    return new_slice
            

    

In [39]:
for _ in range(6):
    z_min, w_min = min(z_w_to_slice.keys())
    z_max, w_max = max(z_w_to_slice.keys())
    

    new_z_ws = itertools.product(range(z_min-1, z_max+2), repeat=2)
    for z, w in new_z_ws:
        if (z, w) not in z_w_to_slice:
            z_w_to_slice[(z, w)] = np.full(z_w_to_slice[(0, 0)].shape, ".")
    
    new_z_w_to_slice = {(z, w): update_layer(z, w) for z, w in z_w_to_slice}
    z_w_to_slice = new_z_w_to_slice
    

In [40]:
total = 0
for key in z_w_to_slice:
    total +=  np.sum(z_w_to_slice[key] == "#")

In [41]:
total

1696